In [1]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 23.5 MB/s eta 0:00:00


In [3]:
import pulp
import numpy as np

# Function to solve the Traveling Salesman Problem (TSP)
def tsp(data, method='heuristic'):
    if len(data) <= 3:
        method = 'exact'
    if method == 'heuristic':
        return tsp_heuristic(data)
    elif method == 'exact':
        return tsp_exact(data)
    else:
        raise ValueError("Method must be either 'heuristic' or 'exact'")

def tsp_heuristic(data):
    G = build_graph_from_matrix(data)
    MSTree = minimum_spanning_tree(G)
    odd_vertexes = find_odd_vertexes(MSTree, G)
    augmented_MST = minimum_weight_matching(G, odd_vertexes)
    eulerian_tour = find_eulerian_tour(augmented_MST, G)
    return create_hamiltonian_circuit(eulerian_tour, G)

def tsp_exact(data):
    n = len(data)
    problem = pulp.LpProblem("TSP", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n) for j in range(n) if i != j], 0, 1, pulp.LpBinary)
    problem += pulp.lpSum([x[(i, j)] * data[i][j] for i in range(n) for j in range(n) if i != j])
    for i in range(n):
        problem += pulp.lpSum([x[(i, j)] for j in range(n) if i != j]) == 1
        problem += pulp.lpSum([x[(j, i)] for j in range(n) if i != j]) == 1
    u = pulp.LpVariable.dicts("u", range(n), 0, n-1, pulp.LpInteger)
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                problem += u[i] - u[j] + n * x[(i, j)] <= n - 1
    problem.solve()
    if pulp.LpStatus[problem.status] != "Optimal":
        raise Exception(f"No optimal solution found. Status: {pulp.LpStatus[problem.status]}")
    tour = extract_tour(x, n)
    total_distance = calculate_total_distance(tour, data)
    return total_distance, tour

def extract_tour(x, n):
    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in range(n):
            if j != current_city and pulp.value(x[(current_city, j)]) == 1:
                tour.append(j)
                current_city = j
                break
    return tour

def calculate_total_distance(tour, data):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += data[tour[i]][tour[i + 1]]
    total_distance += data[tour[-1]][tour[0]]
    return total_distance

def build_graph_from_matrix(data):
    n = len(data)
    graph = {i: {} for i in range(n)}
    for i in range(n):
        for j in range(i + 1, n):
            graph[i][j] = graph[j][i] = data[i][j]
    return graph

# Define your dataset as a distance matrix
data_matrix = np.array([[0, 3, 4, 2],
                        [3, 0, 1, 4],
                        [4, 1, 0, 5],
                        [2, 4, 5, 0]])

# Solve TSP
length, path = tsp(data_matrix, "exact")

print(f"Path: {path}")
print(f"Total length: {length}")

# Approximate the upper bound using the approximation factor (3/2)
upper_bound = length * (3/2)
print(f"Approximate Upper Bound optimal path distance: {upper_bound}")


Path: [0, 3, 2, 1]
Total length: 11
Approximate Upper Bound optimal path distance: 16.5


In [4]:
import random
import copy

class GeneticAlgorithmTSP:
    def __init__(self, graph, population_size=10, mutation_rate=0.01, generations=100):
        self.graph = graph
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations

    def greedy_path(self, s, path, visited):
        visited[s] = True
        adj = self.graph[s]
        minimum = float('inf')
        child = -1
        for i in range(len(adj)):
            if not visited[i] and adj[i] < minimum and adj[i] != 0:
                child = i
                minimum = adj[i]
        if child != -1:
            path.append(child)
            self.greedy_path(child, path, visited)

    def shortest_path(self, source):
        size = len(self.graph)
        path = [source]
        visited = [False] * size
        self.greedy_path(source, path, visited)
        print(path)
        return path

    def initial_population(self):
        population =[]
        num_cities = len(self.graph)
        for _ in range(self.population_size):
            # route = list(range(num_cities))
            # random.shuffle(route)
            population.append(path)
        # print(population)
        return population

    def distance(self, city1, city2):
        return self.graph[city1][city2]

    def fitness(self, route):
        total_distance = 0
        for i in range(len(route) - 1):
            total_distance += self.distance(route[i], route[i + 1])
        total_distance += self.distance(route[-1], route[0])
        return 1 / total_distance

    def selection(self, population):
        ranked_routes = [(self.fitness(route), route) for route in population]
        ranked_routes.sort(reverse=True)
        return ranked_routes[0][1]

    def breed(self, parent1, parent2):
        start = random.randint(0, len(parent1) - 1)
        end = random.randint(0, len(parent1) - 1)
        if start > end:
            start, end = end, start

        child = [-1] * len(parent1)
        for i in range(start, end + 1):
            child[i] = parent1[i]

        j = 0
        for i in range(len(parent2)):
            if j == start:
                j = end + 1
            if parent2[i] not in child:
                child[j] = parent2[i]
                j += 1

        return child

    def mutate(self, route):
        for i in range(len(route)):
            if random.random() < self.mutation_rate:
                j = random.randint(0, len(route) - 1)
                route[i], route[j] = route[j], route[i]

    def evolve(self):
        population = self.initial_population()
        for i in range(self.generations):
            best_route = self.selection(population)
            print(f"Generation {i + 1}, Best route: {best_route}")
            new_population = [best_route]
            for _ in range(1, self.population_size):
                parent1_index = random.randint(0, len(population) - 1)
                parent2_index = random.randint(0, len(population) - 1)
                child = self.breed(population[parent1_index], population[parent2_index])
                self.mutate(child)
                new_population.append(child)
            population = copy.deepcopy(new_population)

if __name__ == "__main__":
    random.seed(42)  # for reproducibility

    graph = data_matrix

    tsp_ga = GeneticAlgorithmTSP(graph)
    tsp_ga.evolve()



Generation 1, Best route: [0, 3, 2, 1]
Generation 2, Best route: [0, 3, 2, 1]
Generation 3, Best route: [0, 3, 2, 1]
Generation 4, Best route: [0, 3, 2, 1]
Generation 5, Best route: [0, 3, 2, 1]
Generation 6, Best route: [0, 3, 2, 1]
Generation 7, Best route: [0, 3, 2, 1]
Generation 8, Best route: [2, 3, 0, 1]
Generation 9, Best route: [2, 3, 0, 1]
Generation 10, Best route: [3, 0, 2, 1]
Generation 11, Best route: [3, 0, 2, 1]
Generation 12, Best route: [3, 0, 2, 1]
Generation 13, Best route: [3, 0, 2, 1]
Generation 14, Best route: [3, 0, 2, 1]
Generation 15, Best route: [3, 0, 2, 1]
Generation 16, Best route: [3, 0, 2, 1]
Generation 17, Best route: [3, 0, 2, 1]
Generation 18, Best route: [3, 0, 2, 1]
Generation 19, Best route: [3, 1, 2, 0]
Generation 20, Best route: [3, 1, 2, 0]
Generation 21, Best route: [3, 1, 2, 0]
Generation 22, Best route: [3, 1, 2, 0]
Generation 23, Best route: [3, 1, 2, 0]
Generation 24, Best route: [3, 1, 2, 0]
Generation 25, Best route: [3, 1, 2, 0]
Generatio